In [1]:
from torchvision import datasets
# MNISTデータの取得
# https://pytorch.org/vision/stable/generated/torchvision.datasets.MNIST.html#torchvision.datasets.MNIST
# 学習用
train_dataset = datasets.MNIST(
    './data/torch',               # データの保存先
    train = True,           # 学習用データを取得する
    download = True,        # データが無い時にダウンロードする
    # transform = transform   # テンソルへの変換など
    )
# 評価用
test_dataset = datasets.MNIST(
    './data/torch', 
    train = False,
    # transform = transform
    )

In [2]:
import boto3
from botocore.exceptions import ClientError
from botocore.config     import Config
s3 = boto3.resource(
	service_name          = "s3",
	endpoint_url          = "http://minio:9000",
	aws_access_key_id     = 'minioadminuser',
	aws_secret_access_key = 'minioadminpassword',
	config                = Config(proxies={'http': '', 'https': ''}))
try:
	bucket = s3.create_bucket(Bucket='digit-images')
except ClientError as e:
	if e.response['Error']['Code'] in ('BucketAlreadyExists', 'BucketAlreadyOwnedByYou'):
		bucket = s3.Bucket('digit-images')
	else:
		print(f'Unknown exception.\n\t ' + e.response['Error']['Code'])
		raise

import re
import MySQLdb
mysql = MySQLdb.connect(
		user='root',
		password='root_password',
		host='mysql',
		port=3306,
		database='DigitImages'
	)
mysqlCursor = mysql.cursor()
tableName='MNIST'
mysqlCursor.execute(f'create table IF NOT EXISTS {tableName}(id INT AUTO_INCREMENT primary key, relpath varchar(100), label INT, date DATETIME, is_used BOOLEAN)')


0

In [5]:
indices = [0] * 10
dataBase = []

import os
import matplotlib.pyplot as plt
total=len(train_dataset)
uploaded=0
for image, label in train_dataset:
	basename = f'train_{label}_{indices[label]}.jpg'
	indices[label] += 1
	localpath=os.path.join('./data/',basename)
	image.save(localpath)
	bucket.upload_file(localpath, tableName+'/'+basename)
	mysqlCursor.execute(f"INSERT INTO {tableName}(relpath,label,date,is_used) VALUES('{basename}',{label},NULL,false)")
	os.remove(localpath)
	uploaded += 1
	print(f'\r {uploaded} / {total}', end="")
mysql.commit()

total=len(test_dataset)
uploaded=0
for image, label in test_dataset:
	basename = f'test_{label}_{indices[label]}.jpg'
	indices[label] += 1
	localpath=os.path.join('./data/', basename)
	image.save(localpath)
	bucket.upload_file(localpath, tableName+'/'+basename)
	mysqlCursor.execute(f"INSERT INTO {tableName}(relpath,label,date,is_used) VALUES('{basename}',{label},NULL,false)")
	os.remove(localpath)
	uploaded += 1
	print(f'\r {uploaded} / {total}', end="")
mysql.commit()

mysql.close()

 10000 / 10000